In [1]:
import torch
from torchvision import transforms
import numpy as np
from tqdm import tqdm
from PIL import Image
from RNN.RNN1 import *


In [2]:
def benchmark_it_RNN(data_loader, model, device):
    differences = []
    model.eval() 
    
    with torch.no_grad():
        hidden_state = None 

        for images, labels in tqdm(data_loader):
            images, labels = images.to(device), labels.to(device)
            predictions, hidden_state = model(images, hidden_state)

            hidden_state = tuple(h.detach() for h in hidden_state)

            predictions = predictions.squeeze(-1)

            for pred, true_label in zip(predictions.view(-1), labels.view(-1)):
                difference = abs((pred - true_label).item())
                differences.append(difference)

    benchmark_score = np.mean(np.abs(differences))

    return benchmark_score, differences

In [4]:
model_path = "lane_detection_rnn_new_RNN1.pth"
input_shape = (3, 480, 640) 
rnn_hidden_size = 128

device = "cpu" #torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = load_RNN_model("RNN/models/"+model_path, input_shape, device, rnn_hidden_size)

images_dir = "datasets/benchmark/images"
labels_dir = "datasets/benchmark/labels"
transform = get_transform()

seq_length = 20 
batch_size = 1 

benchmark_loader = get_sequential_dataloader(
    images_dir, labels_dir, batch_size=batch_size, seq_length=seq_length,
    train_fraction=0.0, val_fraction=0.0, test_fraction=1.0 
)[2]


RNN model loaded from 'RNN/models/lane_detection_rnn_new_RNN1.pth'


In [5]:
benchmark_score, differences = benchmark_it_RNN(benchmark_loader, model, device)

print(f"benchmark score (average difference): {benchmark_score:.8f}")


100%|██████████| 50/50 [00:25<00:00,  1.96it/s]

benchmark score (average difference): 0.01419988
